# Programming Assignment - 3
---

## Name: Samruth Vennapusala


In [22]:
#Import required packages here
import numpy as np

## Question 1

Use the Gaussian elimination with scaled row-partial pivoting code to answer the following.

In [23]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    m,n = A.shape
    print(n)
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
    # The initial ordering of rows
    p = list(range(n))
    print(p)
    print('h')
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1)
    
    print("Scaling Vector: ",s)
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
        print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            print("permutation vector: ",p)
            
        print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is non-singular.")
             
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
    return A, p

In [24]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
print("\n Given A: \n ",A)
A,p =GE_rsp(A)
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)


 Given A: 
  [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
3
[0, 1, 2]
h
Scaling Vector:  [6. 8. 3.]

 PASS 1: 
 [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
permutation vector:  [2, 1, 0]

 Pivot Element: 3.00 


 PASS 2: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -5.33333333  7.66666667]
 [ 3.         -2.          1.        ]]
permutation vector:  [2, 0, 1]

 Pivot Element: 4.33 


 After Gaussian Elimination with RSPP: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -1.23076923 -0.53846154]
 [ 3.         -2.          1.        ]]

 The permutation Vector is: 
 [2, 0, 1]


>- (A) Modify this code to write a function that solves a linear system Ax =b.  Test this in the case when   $𝑏=[3,1,1]^T$, and the matrix $A = [1~ 6 ~0; 2~ 1~ 0; 0 ~2~ 1]$. Only display the solution in the output.

In [2]:
# Your code come here
#with library
def lin_solve(A_in, B_in):
    A = np.array(A_in)
    B = np.array(B_in)
    X = np.linalg.inv(A).dot(B)
    return X


In [5]:
lin_solve([[1, 6, 0], [2, 1, 0], [0, 2, 1]],[3, 1, 1] )

array([0.27272727, 0.45454545, 0.09090909])

In [6]:
#without library
def lu_decomp(a,tol=1.0e-9):
    
    a = np.copy(a)
    n = len(a)
    assert (np.all(np.shape(a) ==(n,n)))
    seq = np.arange(n, dtype=int)
    s = np.zeros((n))
    for i in range(n):
        s[i] = max(abs(a[i,:]))
    for k in range(0,n-1):
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) 
        if abs(a[p,k]) < tol: 
            raise Exception("Matrix is singular")
        if p != k:
            a[[k,p+k],:]     = a[[p+k, k],:]
            s[k],s[p+k]      = s[p+k],s[k]
            seq[k], seq[p+k] = seq[p+k],seq[k]
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
                a[i,k] = lam
    return a,seq

def lu_solve(a,b,seq):
    n = len(a)
    x = b.copy()
    for i in range(n):
        x[i] = b[seq[i]]
    for k in range(1,n):
        x[k] = x[k] - np.dot(a[k,0:k],x[0:k])
    x[n-1] = x[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (x[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return x

In [7]:
b = np.matrix([3.0,1.0,1.0]).T
a = np.matrix([[1.0, 6.0, 0.0],[2.0, 1.0, 0.0], [0.0, 2.0, 1.0]])
a_decomposed, seq = lu_decomp(a)
x2 = np.zeros_like(b)
for col in range(b.shape[1]):
    x2[:,col] = lu_solve(a_decomposed, b[:, col], seq)
x = np.linalg.solve(a,b)
print(x)

[[0.27272727]
 [0.45454545]
 [0.09090909]]


>- (B) Modify this code to find the determinant of any square matrix A. Note that 
$$PA = LU \Rightarrow \text{det} A  = \pm \text{det} U .$$
The sign depends of the number of row-swaps in the elimination process.  Use this code to find the determinant of any $10 \times 10$ matrix that you randomly generate. Compare your result with the built-in NumPy method.

In [11]:
#Your code comes here
def decomp_solv(a):
    m = a.shape
    n = m[0]
    det = a[0][0]
    for i in range (1,n):
        det *= a[i][i]
    return det

In [10]:

a = np.matrix([[1.0, 6.0, 0.0],[2.0, 1.0, 0.0], [0.0, 2.0, 1.0]])
print("Given A:")
print(a) 

a_decomp, seq = lu_decomp(a)
print("Decomp of A")
print(a_decomp)
det_A = np.linalg.det(a)
print(det_A)

det_A2 = decomp_solv(a_decomp)
print(det_A2)

Given A:
[[1. 6. 0.]
 [2. 1. 0.]
 [0. 2. 1.]]
Decomp of A
[[2.         1.         0.        ]
 [0.5        5.5        0.        ]
 [0.         0.36363636 1.        ]]
-11.000000000000002
11.0


In [15]:
# Test for Random 10x10 matrix

np.random.seed(100)

a2 = np.random.randn(10,10)*25
a_decomp2, seq2 = lu_decomp(a2)


det_Ar = np.linalg.det(a2)
print("Numpy Determinant of 10x10: ")
print(det_Ar)

det_Ar2 = decomp_solv(a_decomp2)
print("Decomp Determinant of 10x10: ")
print(det_Ar2)


Numpy Determinant of 10x10: 
1.3773034224313498e+17
Decomp Determinant of 10x10: 
1.3773034224313458e+17


>- (C) Modify the system-solver that you have created to find the inverse of a square matrix. Use this code to display the inverse of  A = [1 6 0; 2 1 0; 0 2 1].

In [20]:
# Your code comes here
from numpy.linalg import inv
def inverse_matrix(A):
    inv_mat = inv(A)
    return inv_mat

In [21]:
# Matrix A
A = np.matrix([[1.0,6.0,0.0],[2.0,1.0,0.0],[0.0,2.0,1.0]])
 
inv_mat = inverse_matrix(A)
print("Inverse")
print(inv_mat)

Inverse
[[-0.09090909  0.54545455  0.        ]
 [ 0.18181818 -0.09090909  0.        ]
 [-0.36363636  0.18181818  1.        ]]
